## Part 1: Explore the Data

Import the data and use Pandas to learn more about the dataset.

In [1]:
import pandas as pd

df = pd.read_csv('Resources/client_dataset.csv')

df.head()

,first,last,job,phone,email,client_id,order_id,order_date,order_week,order_year,item_id,category,subcategory,unit_price,unit_cost,unit_weight,qty,line_number
0,Donald,Harding,Immunologist,793-904-7725x39308,harding.donald.7185@sullivan.com,58515,8953482,2023-04-28,17,2023,EUD29711-63-6U,decor,wall art,1096.80,762.71,7.50,105,1
1,Tiffany,Myers,Music therapist,201.442.4543x942,myers.t.6537@ferguson-johnson.net,37609,8069089,2023-05-19,20,2023,XDA18116-89-4A,consumables,pens,24.95,15.09,1.49,21,0
2,Shannon,Watson,Immunologist,687.737.9424x8503,swatson8146@payne.net,57113,1902144,2023-01-29,4,2023,ABE59463-05-7E,software,project management,13.52,7.86,1.68,39,6
3,Nathan,Baker,Accounting technician,827-788-8123x012,bakernathan@benson.com,46554,9031802,2023-04-25,17,2023,ZMM00836-65-0C,consumables,pens,36.42,24.85,1.23,29,3
4,Christina,Schwartz,Chiropractor,265-829-3643,christinaschwartz9252@mcconnell.com,92089,1322274,2023-05-28,21,2023,BZX55559-12-3X,consumables,misc,195.10,108.17,46.43,20,1


In [2]:
print("Column Names:")
print(df.columns)



Column Names:
Index(['first', 'last', 'job', 'phone', 'email', 'client_id', 'order_id',
       'order_date', 'order_week', 'order_year', 'item_id', 'category',
       'subcategory', 'unit_price', 'unit_cost', 'unit_weight', 'qty',
       'line_number'],
      dtype='object')


In [37]:
print("Basic Statistics:")
print(df.describe())



Basic Statistics:
          client_id      order_id    order_week    order_year    unit_price  \
count  54639.000000  5.463900e+04  54639.000000  54639.000000  54639.000000   
mean   54837.869416  5.470190e+06     11.359139   2022.993064    136.267207   
std    25487.438231  2.599807e+06      7.023499      0.082997    183.873135   
min    10033.000000  1.000886e+06      1.000000   2022.000000      0.010000   
25%    33593.000000  3.196372e+06      6.000000   2023.000000     20.800000   
50%    53305.000000  5.496966e+06     11.000000   2023.000000     68.310000   
75%    78498.000000  7.733869e+06     17.000000   2023.000000    173.160000   
max    99984.000000  9.998480e+06     52.000000   2023.000000   1396.230000   

          unit_cost   unit_weight           qty   line_number      subtotal  \
count  54639.000000  54639.000000  5.463900e+04  54639.000000  5.463900e+04   
mean      99.446073      5.004116  5.702646e+02      2.979667  7.710286e+04   
std      133.164267      5.326599

In [41]:

unique_values = df.nunique()
print("Unique Values:")
print(unique_values)


Unique Values:
first                     360
last                      477
job                       498
phone                     999
email                     999
client_id                 999
order_id                10000
order_date                150
order_week                 23
order_year                  2
item_id                   500
category                    5
subcategory                18
unit_price                490
unit_cost                 487
unit_weight               393
qty                      2911
line_number                10
subtotal                41717
total_weight            30079
shipping_price          29758
shipping_cost           29758
total_price             42514
total_price_with_tax    42700
line_cost               41343
line_profit             40918
dtype: int64


In [42]:
top_categories = df['category'].value_counts().head(3)
print("Top three item categories with the most entries:")
print(top_categories)


Top three item categories with the most entries:
category
consumables    23538
furniture      11915
software        8400
Name: count, dtype: int64


In [2]:
most_entries_category = df['category'].value_counts().idxmax()

category_df = df[df['category'] == most_entries_category]

most_entries_subcategory = category_df['subcategory'].value_counts().head(1)

print("Subcategory with the most entries in the category with the most entries:")
print(most_entries_subcategory)


Subcategory with the most entries in the category with the most entries:
subcategory
bathroom supplies    6424
Name: count, dtype: int64


In [3]:
top_clients = df['client_id'].value_counts().head()
print("Top five clients with the most entries:")
print(top_clients)


Top five clients with the most entries:
client_id
33615    220
66037    211
46820    209
38378    207
24741    207
Name: count, dtype: int64


In [5]:
top_client_ids = top_clients.index.tolist()
print(top_client_ids)


[33615, 66037, 46820, 38378, 24741]


In [46]:
total_units_top_client = df[df['client_id'] == top_client_ids[0]]['qty'].sum()
print("Total units ordered by the client with the most entries:", total_units_top_client)


Total units ordered by the client with the most entries: 64313


## Part 2: Transform the Data
Do we know that this client spent the more money than client 66037? If not, how would we find out? Transform the data using the steps below to prepare it for analysis.

In [10]:
# Create a column that calculates the 
# subtotal for each line using the unit_price
# and the qty
df['subtotal'] = df['unit_price'] * df['qty']


In [19]:
# Create a column for shipping price.
# Assume a shipping price of $7 per pound
# for orders over 50 pounds and $10 per
# pound for items 50 pounds or under.

def calculate_shipping_cost(total_weight):
    if total_weight > 50:
        return total_weight * 7  
    else:
        return total_weight * 10  

df['shipping_price'] = df['total_weight'].apply(calculate_shipping_cost)

print(df)



           first       last                      job               phone  \
0         Donald    Harding             Immunologist  793-904-7725x39308   
1        Tiffany      Myers          Music therapist    201.442.4543x942   
2        Shannon     Watson             Immunologist   687.737.9424x8503   
3         Nathan      Baker    Accounting technician    827-788-8123x012   
4      Christina   Schwartz             Chiropractor        265-829-3643   
...          ...        ...                      ...                 ...   
54634     Lauren      Reese           Radio producer  977.877.5272x11382   
54635    Derrick      Moore                 Musician        358.661.5483   
54636     Monica  Gutierrez         Graphic designer    294.805.9100x339   
54637      Wanda    Solomon             Toxicologist       (311)767-4924   
54638     Dwayne       Bell  Chief Financial Officer   606-619-7667x9508   

                                     email  client_id  order_id  order_date  \
0       

In [18]:
# Create a column for the total price
# using the subtotal and the shipping price
# along with a sales tax of 9.25%

sales_tax_rate = 0.0925

df['total_price'] = df['subtotal'] + df['shipping_price']
df['total_price_with_tax'] = df['total_price'] * (1 + sales_tax_rate)

print(df)

           first       last                      job               phone  \
0         Donald    Harding             Immunologist  793-904-7725x39308   
1        Tiffany      Myers          Music therapist    201.442.4543x942   
2        Shannon     Watson             Immunologist   687.737.9424x8503   
3         Nathan      Baker    Accounting technician    827-788-8123x012   
4      Christina   Schwartz             Chiropractor        265-829-3643   
...          ...        ...                      ...                 ...   
54634     Lauren      Reese           Radio producer  977.877.5272x11382   
54635    Derrick      Moore                 Musician        358.661.5483   
54636     Monica  Gutierrez         Graphic designer    294.805.9100x339   
54637      Wanda    Solomon             Toxicologist       (311)767-4924   
54638     Dwayne       Bell  Chief Financial Officer   606-619-7667x9508   

                                     email  client_id  order_id  order_date  \
0       

In [20]:
# Create a column for the cost
# of each line using unit cost, qty, and
# shipping price (assume the shipping cost
# is exactly what is charged to the client).

df['line_cost'] = df['unit_cost'] * df['qty']

df['total_price'] = df['line_cost'] + df['shipping_price']

print(df)

           first       last                      job               phone  \
0         Donald    Harding             Immunologist  793-904-7725x39308   
1        Tiffany      Myers          Music therapist    201.442.4543x942   
2        Shannon     Watson             Immunologist   687.737.9424x8503   
3         Nathan      Baker    Accounting technician    827-788-8123x012   
4      Christina   Schwartz             Chiropractor        265-829-3643   
...          ...        ...                      ...                 ...   
54634     Lauren      Reese           Radio producer  977.877.5272x11382   
54635    Derrick      Moore                 Musician        358.661.5483   
54636     Monica  Gutierrez         Graphic designer    294.805.9100x339   
54637      Wanda    Solomon             Toxicologist       (311)767-4924   
54638     Dwayne       Bell  Chief Financial Officer   606-619-7667x9508   

                                     email  client_id  order_id  order_date  \
0       

In [29]:
# Create a column for the profit of
# each line using line cost and line price
df['line_profit'] = df['subtotal'] - df['line_cost']

print(df)



           first       last                      job               phone  \
0         Donald    Harding             Immunologist  793-904-7725x39308   
1        Tiffany      Myers          Music therapist    201.442.4543x942   
2        Shannon     Watson             Immunologist   687.737.9424x8503   
3         Nathan      Baker    Accounting technician    827-788-8123x012   
4      Christina   Schwartz             Chiropractor        265-829-3643   
...          ...        ...                      ...                 ...   
54634     Lauren      Reese           Radio producer  977.877.5272x11382   
54635    Derrick      Moore                 Musician        358.661.5483   
54636     Monica  Gutierrez         Graphic designer    294.805.9100x339   
54637      Wanda    Solomon             Toxicologist       (311)767-4924   
54638     Dwayne       Bell  Chief Financial Officer   606-619-7667x9508   

                                     email  client_id  order_id  order_date  \
0       

## Part 3: Confirm your work
You have email receipts showing that the total prices for 3 orders. Confirm that your calculations match the receipts. Remember, each order has multiple lines.

Order ID 2742071 had a total price of \$152,811.89

Order ID 2173913 had a total price of \$162,388.71

Order ID 6128929 had a total price of \$923,441.25


In [30]:
# Check your work using the totals above
order_ids_to_check = [2742071, 2173913, 6128929]
filtered_orders = df[df['order_id'].isin(order_ids_to_check)]

total_prices = filtered_orders.groupby('order_id')['total_price'].sum()

print("Calculated Total Prices:")
print(total_prices)

Calculated Total Prices:
order_id
2173913    130500.17
2742071    124058.01
6128929    625740.71
Name: total_price, dtype: float64


## Part 4: Summarize and Analyze
Use the new columns with confirmed values to find the following information.

In [32]:
# How much did each of the top 5 clients by quantity
# spend? Check your work from Part 1 for client ids.

top_clients_by_quantity = df.groupby('client_id')['qty'].sum().nlargest(5)

top_clients_orders = df[df['client_id'].isin(top_clients_by_quantity.index)]

total_spent_by_client = top_clients_orders.groupby('client_id')['total_price'].sum()

print("Total amount spent by each of the top 5 clients by quantity:")
print(total_spent_by_client)

In [34]:
# Create a summary DataFrame showing the totals for the
# for the top 5 clients with the following information:
# total units purchased, total shipping price,
# total revenue, and total profit. Sort by total profit.

grouped_by_client = df.groupby('client_id')

summary_df = pd.DataFrame({
    'Total Units Purchased': grouped_by_client['qty'].sum(),
    'Total Shipping Price': grouped_by_client['shipping_price'].sum(),
    'Total Revenue': grouped_by_client['total_price'].sum(),
    'Total Profit': grouped_by_client['line_profit'].sum()
})

top_5_clients_summary = summary_df.nlargest(5, 'Total Profit')

print("Summary DataFrame for the top 5 clients:")
print(top_5_clients_summary)

Summary DataFrame for the top 5 clients:
           Total Units Purchased  Total Shipping Price  Total Revenue  \
client_id                                                               
78965                    4033302            5340756.88   4.085708e+08   
85931                     644423           10122549.19   1.281229e+08   
24741                     239862            5126448.37   4.568890e+07   
38473                     402770            4391003.73   1.402008e+08   
86721                    1010426           64335727.72   1.382916e+08   

           Total Profit  
client_id                
78965      1.918170e+08  
85931      8.919246e+07  
24741      2.961443e+07  
38473      2.781066e+07  
86721      2.762721e+07  


In [35]:
# Format the data and rename the columns
# to names suitable for presentation.
# Currency should be in millions of dollars.

top_5_clients_summary_millions = top_5_clients_summary / 1000000

top_5_clients_summary_millions.columns = ['Total Units Purchased (Millions)', 
                                        'Total Shipping Price (Millions)', 
                                        'Total Revenue (Millions)', 
                                        'Total Profit (Millions)']

print("Formatted and Renamed Summary DataFrame for the top 5 clients:")
print(top_5_clients_summary_millions)



Formatted and Renamed Summary DataFrame for the top 5 clients:
           Total Units Purchased (Millions)  Total Shipping Price (Millions)  \
client_id                                                                      
78965                              4.033302                         5.340757   
85931                              0.644423                        10.122549   
24741                              0.239862                         5.126448   
38473                              0.402770                         4.391004   
86721                              1.010426                        64.335728   

           Total Revenue (Millions)  Total Profit (Millions)  
client_id                                                     
78965                    408.570828               191.816971  
85931                    128.122873                89.192461  
24741                     45.688900                29.614434  
38473                    140.200845                27.810662 

In [36]:
# Sort the updated data by "Total Profit" form highest to lowest
top_5_clients_summary_sorted = top_5_clients_summary_millions.sort_values(by='Total Profit (Millions)', ascending=False)

print("Sorted Summary DataFrame by Total Profit (from highest to lowest):")
print(top_5_clients_summary_sorted)


Sorted Summary DataFrame by Total Profit (from highest to lowest):
           Total Units Purchased (Millions)  Total Shipping Price (Millions)  \
client_id                                                                      
78965                              4.033302                         5.340757   
85931                              0.644423                        10.122549   
24741                              0.239862                         5.126448   
38473                              0.402770                         4.391004   
86721                              1.010426                        64.335728   

           Total Revenue (Millions)  Total Profit (Millions)  
client_id                                                     
78965                    408.570828               191.816971  
85931                    128.122873                89.192461  
24741                     45.688900                29.614434  
38473                    140.200845                27.810